# #5. Black-Litterman model

#### #5.9 블랙-리터만 모델 최적화 (p 199)

Optimization condition for BL model:

\begin{align*}
    \max \Sigma_{i=1}^n \mu BL_i &- \frac{\lambda}{2} \Sigma_{i=1}^n \Sigma_{j=1}^n \sigma_{ij} w_i w_j \\
    \text{Subject to} \quad \Sigma_{i=1}^n w_i &=
    
     1, w_i \geq 0. i=1, 2, 3, \ldots, n
\end{align*}

; this enables us to compute the weights $w_i$ for asset allocation.

아래는 Sharpe 비율을 최대로하는 tangency portfolio 최적화를 수행하는 코드다:

In [1]:
import matplotlib.pylab as plt
import numpy as np
from numpy.linalg import inv
import pandas as pd
from pandas_datareader import data as web
from scipy.optimize import minimize

In [2]:
# 최적의 비중 계산을 위한 목적함수
# obj. funciton for optimizing weights
def solveWeights(R, C, rf):
    def obj(W, R, C, rf):
        mean = sum (R * W)
        var = np.dot(np.dot(W, C), W)
        
        # Sharpe's ratio as utility function: want to maximize
        util = (mean -rf) / np.sqrt(var)
        return 1 / util
    
    # number of assets
    n = len(R)
    
    # initial weights
    W = np.ones([n]) / n
    
    # doesn't allow short-selling nor borrowing
    # 비중 범위는 0-100% 사이: 공매도나 차입조건이 없음
    bnds = [(0., 1.) for i in range(n)]
    
    # constraints: sum of weights = 1
    cons = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.})

    # W는 초기값, (R, C, rf)는 추가 인수
    res = minimize(obj, W, (R, C, rf), method='SLSQP', constraints=cons, bounds=bnds)

    # res.success: 최적화(minimize())가 성공적으로 수행되었는지를 나타내는 Boolean 값
    # False이면 res.message로 그 원인을 알 수 있다
    if not res.success:
        raise BaseException(res.message)
    
    return res.x

In [4]:
# example 1
R = np.array([0.05, 0.08])  # 자산 기대수익률
C = np.array([[0.01, 0.02],
              [0.02, 0.04]])  # 자산 공분산
rf = 0.02  # 무위험 이자율
weights = solveWeights(R, C, rf)  # 최적 비중 계산

print("Optimal Weights:", weights)

Optimal Weights: [0.5 0.5]


In [5]:
# example 2
R = [0.05, 0.08, 0.12]  # 자산의 기대수익률
C = [[0.04, 0.02, 0.01],
     [0.02, 0.09, 0.03],
     [0.01, 0.03, 0.06]]  # 자산들 간의 공분산 행렬
rf = 0.03  # 무위험 이자율

weights = solveWeights(R, C, rf)
print(weights)

[0.07040263 0.02735284 0.90224453]


In [9]:
# 무위험 수익률, 수익률, Cov로 효율적 투자선 계산
# calculating Efficient Frontier via risk-less rate, return rate and the Covariance
def solveFrontier(R, C, rf):

    # 최적 비중 계산을 위한 목적함수
    def obj(W, R, C, r):

        # 주어진 수익률에서 분산을 최소화하는 비중 계산
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)

        # 최적화 제약조건 페널티
        penalty = 100 * abs(mean - r)
        return var + penalty
    
    # eff. frontier의 평균과 분산이 될 list
    frontier_mean, frontier_var = [ ], [ ]
    n = len(R)
    
    # 수익률 최저에서 최대 사이에서 목표 수익률(r)을 다양하게 설정하여 포트폴리오 최적화를 반복
    for r in np.linspace(min(R), max(R), num=20):
        W = np.ones([n]) / n
        bnds = [(0, 1) for i in range(n)]
        cons = [{'type': 'eq', 'fun': lambda W: sum(W) - 1.}]

        res = minimize(obj, W, (R, C, r), method='SLSQP', constraints=cons, bounds=bnds)

        if not res.success:
            raise BaseException(res.message)
        
        frontier_mean.append(r)
        frontier_var.append(np.dot(np.dot(res.x, C), res.x))

    return np.array(frontier_mean), np.array(frontier_var)

위 코드에서 obj. f'tn의 penalty 항(이를 minimize하길 원한다)을 주목해보자:

이러한 패널티는 효율적인 포트폴리오 최적화를 위해 사용된다. 목표 수익률과 실제 포트폴리오의 수익률이 일치하는 경우에는 패널티가 없다. 그러나 목표 수익률과 포트폴리오 수익률 간에 차이가 클수록 패널티가 증가한다.

이를 통해, 목표 수익률과 가능한 작은 분산을 동시에 달성하는 포트폴리오를 찾기 위해 최적화 과정에서 목표 수익률에 가까운 포트폴리오를 선호하도록 유도한다. 따라서 목적함수에 패널티 항을 추가하여 수익률과 분산의 트레이드오프를 조정하고 최적의 포트폴리오를 찾을 수 있게 된다.

In [10]:
# example
R = np.array([0.08, 0.12, 0.15])  # 수익률
C = np.array([[0.02, 0.005, 0.01], 
              [0.005, 0.03, 0.02], 
              [0.01, 0.02, 0.04]])  # 공분산
rf = 0.05  # 무위험 이자율

# Efficient Frontier 계산
frontier_mean, frontier_var = solveFrontier(R, C, rf)

# 결과 출력
for mean, var in zip(frontier_mean, frontier_var):
    print(f"Mean Return: {mean:.4f}, Variance: {var:.4f}")

Mean Return: 0.0800, Variance: 0.0200
Mean Return: 0.0837, Variance: 0.0176
Mean Return: 0.0874, Variance: 0.0158
Mean Return: 0.0911, Variance: 0.0148
Mean Return: 0.0947, Variance: 0.0144
Mean Return: 0.0984, Variance: 0.0152
Mean Return: 0.1021, Variance: 0.0146
Mean Return: 0.1058, Variance: 0.0151
Mean Return: 0.1095, Variance: 0.0158
Mean Return: 0.1132, Variance: 0.0167
Mean Return: 0.1168, Variance: 0.0178
Mean Return: 0.1205, Variance: 0.0192
Mean Return: 0.1242, Variance: 0.0208
Mean Return: 0.1279, Variance: 0.0225
Mean Return: 0.1316, Variance: 0.0246
Mean Return: 0.1353, Variance: 0.0268
Mean Return: 0.1389, Variance: 0.0292
Mean Return: 0.1426, Variance: 0.0320
Mean Return: 0.1463, Variance: 0.0355
Mean Return: 0.1500, Variance: 0.0400


In [ ]:
# 효율적 포트폴리오 최적화
def optimize_frontier(R, C, rf):

    # tangency portfolio 계산
    W = solveWeights(R, C, rf)

    tan_mean = sum(R * W)
    tan_var = np.dot(np.dot(W, C), W)

    # 효율적 포트폴리오 계산
    eff_mean, eff_var = solveFrontier(R, C, rf)
    
    # dict 타입으로 리턴
    return {'weights':W, 'tan_mean':tan_mean, 'tan_var':tan_var, 'eff_mean':eff_mean, 'eff_var':eff_var}

In [16]:
# 자산에 대한 투자자의 전망 (행렬)과 전망의 기대수익률 행렬
# names은 tickers다
def CreateMatrixPQ(names, views):
    r, c = len(views), len(names)

    # Q: 기대 수익률 행렬
    # views[i][3] = exp. return(기대 수익률): 아래 예시들 참고
    Q = [views[i][3] for i in range(r)]
    
    # 전망 행렬 P을 만들기위한 구성 자산 dict.
    nameToIndex = dict()
    for i, n in enumerate(names):
        nameToIndex[n] = i
    
    # 투자자의 전망 행렬
    P = np.zeros([r, c])
    for i, v in enumerate(views):
        # 가령 전망이 ('MSFT', '>', 'GE', 0.02) 이라면
        # views[i][0] <-- 'MSFT' --> name1
        # views[i][1] <-- '>'
        # views[i][2] <-- 'GE'   --> name2
        # views[i][3] <-- '0.02'
        name1, name2 = views[i][0], views[i][2]
        P[i, nameToIndex[name1]] = +1 if views[i][1] == '>' else -1
        P[i, nameToIndex[name2]] = -1 if views[i][1] == '>' else +1
    
    return np.array(Q), P

In [30]:
# example 1 (p 189)
tickers = ['XOM', 'JPM', 'NFLX', 'JNJ']
views1 = [('XOM', '>', 'JPM', 0.02), ('NFLX', '<', 'JNJ', 0.02)]
Q, P = CreateMatrixPQ(tickers, views1)

print(Q)
print(P)

[0.02 0.02]
[[ 1. -1.  0.  0.]
 [ 0.  0. -1.  1.]]


In [31]:
# example 2 (p 189-190)
tickers = ['MSFT', 'GE']
views2 = [('MSFT', '>', 'GE', 0.02)]
Q, P = CreateMatrixPQ(tickers, views2)

print(Q)
print(P)

[0.02]
[[ 1. -1.]]


In [21]:
# example 3
names = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
views = [['AAPL', '>', 'GOOGL', 0.03],
         ['AMZN', '<', 'MSFT', 0.02],
         ['GOOGL', '>', 'AAPL', 0.01]]

Q, P = CreateMatrixPQ(names, views)

print("기대 수익률 행렬 (Q):")
print(Q)
print("전망 행렬 (P):")
print(P)


기대 수익률 행렬 (Q):
[0.03 0.02 0.01]
전망 행렬 (P):
[[ 1. -1.  0.  0.]
 [ 0.  0. -1.  1.]
 [-1.  1.  0.  0.]]


In [34]:
# read the data
tickers = ['PFE','INTC','NFLX','JPM','XOM','GOOG','JNJ','AAPL','AMZN']
cap = {'PFE':201102000000,'INTC':257259000000,'NFLX':184922000000,
       'JPM':272178000000,'XOM':178228000000,'GOOG':866683000000,
       'JNJ':403335000000,'AAPL':1208000000000,'AMZN':1178000000000
      }

prices, caps = [ ], [ ]

import yfinance as yf

for s in tickers:
    # pxclose = web.DataReader(s, data_source='yahoo', start='01-01-2018', end='31-12-2019')['Adj Close']
    pxclose = yf.download(s, start='2018-01-01', end='2019-12-31')['Adj Close']
    prices.append(list(pxclose))
    caps.append(cap[s])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [35]:
pxclose.head()

Date
2018-01-02    59.450500
2018-01-03    60.209999
2018-01-04    60.479500
2018-01-05    61.457001
2018-01-08    62.343498
Name: Adj Close, dtype: float64

In [36]:
pxclose

Date
2018-01-02    59.450500
2018-01-03    60.209999
2018-01-04    60.479500
2018-01-05    61.457001
2018-01-08    62.343498
                ...    
2019-12-23    89.650002
2019-12-24    89.460503
2019-12-26    93.438499
2019-12-27    93.489998
2019-12-30    92.344498
Name: Adj Close, Length: 502, dtype: float64

In [41]:
prices

[[28.254047393798828,
  28.463388442993164,
  28.52542495727539,
  28.579696655273438,
  28.261796951293945,
  28.230789184570312,
  28.277307510375977,
  28.347089767456055,
  28.331588745117188,
  28.37809944152832,
  28.827808380126953,
  28.68048667907715,
  28.641725540161133,
  28.63397216796875,
  28.548677444458008,
  28.63397216796875,
  28.8665828704834,
  30.246719360351562,
  30.25447654724121,
  29.308536529541016,
  28.719266891479492,
  28.82098960876465,
  28.648836135864258,
  27.130708694458008,
  27.608051300048828,
  27.373289108276367,
  26.31686019897461,
  26.731611251831055,
  27.12287712097168,
  27.341989517211914,
  27.537622451782227,
  27.944551467895508,
  28.374948501586914,
  28.179306030273438,
  27.983667373657227,
  27.9680233001709,
  28.374948501586914,
  29.024456024169922,
  28.79751968383789,
  28.414066314697266,
  27.881942749023438,
  28.13235855102539,
  28.265390396118164,
  28.07757568359375,
  28.1167049407959,
  28.562759399414062,
  28.7

In [42]:
caps

[201102000000,
 257259000000,
 184922000000,
 272178000000,
 178228000000,
 866683000000,
 403335000000,
 1208000000000,
 1178000000000]

최적화에 쓰일 값을 미리 계산

In [43]:
n = len(tickers)
W = np.array(caps) / sum(caps)
prices = np.matrix(prices)

rows, cols = prices.shape
returns = np.empty([rows, cols - 1])
for r in range(rows):
    for c in range(cols - 1):
        p0, p1 = prices[r, c], prices[r, c + 1]
        returns[r, c] = (p1 / p0) - 1

# 수익률
expreturns = np.array([])
for r in range(rows):
    expreturns = np.append(expreturns, np.mean(returns[r]))

covars = np.cov(returns)
R = (1 + expreturns) ** 250 - 1  # 연율화 annualize
C = covars * 250

rf = .015

In [44]:
expreturns

array([ 0.00034886,  0.00076697,  0.00127909,  0.00068967, -0.00014778,
        0.00058903,  0.00026973,  0.00126127,  0.00106064])

In [45]:
display(pd.DataFrame({'Return':R, 'Weight (based on market cap)':W}, index=tickers).T)

,PFE,INTC,NFLX,JPM,XOM,GOOG,JNJ,AAPL,AMZN
Return,0.091115,0.211269,0.376532,0.188103,-0.036272,0.158603,0.069750,0.370421,0.303458
Weight (based on market cap),0.042340,0.054163,0.038933,0.057304,0.037524,0.182471,0.084918,0.254331,0.248015


In [46]:
display(pd.DataFrame(C, columns=tickers, index=tickers))

,PFE,INTC,NFLX,JPM,XOM,GOOG,JNJ,AAPL,AMZN
PFE,0.037416,0.021683,0.026074,0.015826,0.015789,0.018568,0.018293,0.019736,0.021604
INTC,0.021683,0.093592,0.054494,0.029861,0.027049,0.039894,0.017541,0.045417,0.042485
NFLX,0.026074,0.054494,0.165585,0.031794,0.027628,0.060322,0.020999,0.053177,0.081516
JPM,0.015826,0.029861,0.031794,0.041864,0.022214,0.028071,0.014437,0.027651,0.029185
XOM,0.015789,0.027049,0.027628,0.022214,0.040405,0.024244,0.014398,0.023460,0.022576
GOOG,0.018568,0.039894,0.060322,0.028071,0.024244,0.068207,0.016347,0.044920,0.054470
JNJ,0.018293,0.017541,0.020999,0.014437,0.014398,0.016347,0.037293,0.015893,0.016443
AAPL,0.019736,0.045417,0.053177,0.027651,0.023460,0.044920,0.015893,0.075312,0.051415
AMZN,0.021604,0.042485,0.081516,0.029185,0.022576,0.054470,0.016443,0.051415,0.090683


In [47]:
"""과거 데이터를 이용한 최적화"""
# mean-var optimization
opt1 = optimize_frontier(R, C, rf)

NameError: name 'optimize_frontier' is not defined

In [48]:
"""블랙-리터만 역최적화"""
mean = sum(R * W)
var = np.dot(np.dot(W, C), W)

# 위험회피계수
lmbda = (mean - rf) / var

# 균형초과수익률
pi = np.dot(np.dot(lmbda, C), W)

In [53]:
print("mean:", mean)
print("var:", var)
print("lambda:", lmbda)
print("pi:", pi)

mean: 0.24371377735968996
var: 0.04432157077826753
lambda: 5.160326525968628
pi: [0.10626377 0.21514542 0.30906682 0.14127229 0.11977304 0.24037653
 0.09381257 0.25431559 0.2880074 ]
